In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoConfig, TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', use_fast=True)
model = TFAutoModelForSequenceClassification.from_pretrained("model")

In [3]:
def predict(text):
    return model(tokenizer.encode(text, return_tensors='tf', max_length=512))[0].numpy()[0][0]

In [4]:
reviews = pd.read_csv("s3://yelp-dataset-pt-9/data/csv/reviews.csv")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/spenceradams/.local/share/virtualenvs/Yelp-ds-xpajUQwb/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-262648aa38b6>", line 1, in <module>
    reviews = pd.read_csv("s3://yelp-dataset-pt-9/data/csv/reviews.csv")
  File "/Users/spenceradams/.local/share/virtualenvs/Yelp-ds-xpajUQwb/lib/python3.7/site-packages/pandas/io/parsers.py", line 676, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/Users/spenceradams/.local/share/virtualenvs/Yelp-ds-xpajUQwb/lib/python3.7/site-packages/pandas/io/parsers.py", line 454, in _read
    data = parser.read(nrows)
  File "/Users/spenceradams/.local/share/virtualenvs/Yelp-ds-xpajUQwb/lib/python3.7/site-packages/pandas/io/parsers.py", line 1133, in read
    ret = self._engine.read(nrows)
  File "/Users/spenceradams/.local/share/virtualenvs/Yelp-ds-xpajUQwb/lib/python3.7/site-p

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
reviews.head()

In [ ]:
reviews.shape

In [ ]:
business_counts = reviews.groupby("business_id").count()